In [ ]:
import pandas as pd
import numpy as np
from transformers import pipeline
from tqdm import tqdm

In [ ]:
books = pd.read_csv('books.csv')

In [ ]:
classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", top_k=None)

In [ ]:
classifier("I am not feeling well")

In [ ]:
books.sample(4)

In [ ]:
ans=classifier(books.description[0].split("."))
ans

In [ ]:
sorted(ans[0],key=lambda x :x['label'])

In [ ]:
emotion_list=[]

In [ ]:
list(map(lambda item: emotion_list.append(item['label']), ans[0]))
emotion_list

In [ ]:
emotion_score= {label :[]  for label in emotion_list} 

In [ ]:
emotion_score

In [ ]:
def get_max_score(predictions):
    per_emotion_score = {label:[] for label in emotion_list}
    for pred in predictions:
        sorted_pred = sorted(pred,key=lambda x :x['label'])
        for index,label in enumerate(emotion_list):
            per_emotion_score[label].append(sorted_pred[index]['score'])
    return {label: np.max(scores) for label,scores in per_emotion_score.items()}

In [ ]:
get_max_score(ans)

In [ ]:
isbn=[]
emotion_score= {label :[]  for label in emotion_list} 

In [ ]:
books_missing=books[~books['description'].isna() &
      ~books['num_pages'].isna() &
      ~books['average_rating'].isna()&
      ~books['published_year'].isna()
]

In [ ]:
books_missing['words_in_description']=books_missing["description"].str.split().str.len()

In [ ]:
books_missing25_words=books_missing[books_missing['words_in_description']>25]

In [ ]:
books_missing25_words.shape

In [ ]:
books_missing['isbn13'].iloc[37]

In [ ]:
pd.read_csv("clean_book.csv").shape

In [ ]:
for i in tqdm(range(0,len(books_missing25_words))):
    isbn.append(books_missing25_words['isbn13'].iloc[i])
    desc = books_missing25_words['description'].iloc[i].split(".")
    predictions = classifier(desc)

    max_score = get_max_score(predictions)

    for label  in emotion_list:
        emotion_score[label].append(max_score[label])

In [ ]:
df_emotion= pd.DataFrame(emotion_score)

In [ ]:
df_emotion['isbn13']=isbn

In [ ]:
df_emotion

In [ ]:
df_new = pd.read_csv("new_book.csv")
df_new.shape

In [ ]:
final_book = pd.merge(df_new,df_emotion,on="isbn13")

In [ ]:
final_book.isna().sum()

In [ ]:
final_book.to_csv("book_with_emotion.csv",index=False)